# The structure of a mesh

To explain the structure of a mesh, we create a helper function:

In [455]:
from polymesh import PointData, PolyData
from polymesh.cells import T3, Q4, L2
import numpy as np


def generate_mesh(to_standard_form:bool=False):

    coords_T3 = np.array([
        [0, 0, 0],
        [1, 0, 0],
        [1, 1, 0],
        [0, 1, 0],
    ], dtype=float)

    topology_T3 = np.array([
        [0, 1, 2],
        [0, 2, 3],
    ], dtype=int)

    coords_Q4 = np.array([
        [2, 0, 0],
        [3, 0, 0],
        [3, 1, 0],
        [2, 1, 0],
    ], dtype=float)

    topology_Q4 = np.array([
        [0, 1, 2, 3],
    ], dtype=int)

    coords_L2 = np.array([
        [1, 0, 0],
        [2, 1, 0],
        [1, 1, 0],
        [2, 0, 0],
    ], dtype=float)

    topology_L2 = np.array([
        [0, 1],
        [2, 3]
    ], dtype=int)

    pd_T3 = PointData(coords=coords_T3)
    cd_T3 = T3(topo=topology_T3)

    pd_Q4 = PointData(coords=coords_Q4)
    cd_Q4 = Q4(topo=topology_Q4)

    pd_L2 = PointData(coords=coords_L2)
    cd_L2 = L2(topo=topology_L2)

    mesh = PolyData()
    mesh["2d", "triangles"] = PolyData(pd_T3, cd_T3)
    mesh["2d", "quads"] = PolyData(pd_Q4, cd_Q4)
    mesh["lines"] = PolyData(pd_L2, cd_L2)
    
    if to_standard_form:
        mesh.to_standard_form()
    
    return mesh


mesh = generate_mesh()

## The database model

A ``PolyData`` object is essentially a dictionary, equipped with one or two Awkward arrays to store data attached to the points and the cells. Instances walk and talk like a dictionary but the behaviour is extended to cover nested definitions.

In [456]:
isinstance(mesh, dict)

True

In [457]:
list(mesh.keys())

['2d', 'lines']

In [458]:
list(mesh.values())

[PolyData({'triangles': PolyData({}), 'quads': PolyData({})}), PolyData({})]

As you can see, the `values` call returns only two items, but the mesh has three blocks. To loop through the subdictionaries (called blocks) with cell data, you can use the ``cellblocks`` method of any ``PolyData`` instance. Every block has an address and a parent except the root object, that has no parent and address.

In [459]:
for block in mesh.cellblocks(inclusive=True):
    print(block.address)

['2d', 'triangles']
['2d', 'quads']
['lines']


The parameter ``inclusive`` means to start parsing the structure of the mesh with the instance the call was made upon. In this case it makes no difference, as the root instance of the mesh has no attached cells.

In [460]:
for block in mesh.pointblocks(inclusive=True):
    print(block.address)

['2d', 'triangles']
['2d', 'quads']
['lines']


Now bring the mesh into a standard form and repeat the above queries:

In [461]:
mesh.to_standard_form()

PolyData({'2d': PolyData({'triangles': PolyData({}), 'quads': PolyData({})}), 'lines': PolyData({})})

In [462]:
for block in mesh.cellblocks(inclusive=True):
    print(block.address)

['2d', 'triangles']
['2d', 'quads']
['lines']


In [463]:
for block in mesh.pointblocks(inclusive=True):
    print(block.address)

[]


An empty list is returned, since the root of the mesh does host a point cloud, but it doesn't have an address. To see if a block has an attached point or cell related data,use the `pointdata` and `celldata` properties (you can also use `mesh.pd` and `mesh.cd`):

In [464]:
mesh = generate_mesh()

In [465]:
mesh["2d", "triangles"].pointdata

<Array [{_x: [0, ..., 0], ...}, ..., {...}] type='4 * {_x: 3 * float64, _ac...'>

In [466]:
type(mesh["2d", "triangles"].pointdata)

polymesh.pointdata.PointData

In [467]:
mesh["2d", "triangles"].celldata

<Array [{_nodes: [0, ...], _id: 0}, {...}] type='2 * {_nodes: 3 * int32, _i...'>

In [468]:
type(mesh["2d", "triangles"].celldata)

polymesh.cells.t3.T3

`PointData` and `CellData` instances are wrapper objects that wrap Awkward arrays. The databases can be accessed using the `db` property:

In [469]:
type(mesh["2d", "triangles"].pointdata.db)

awkward.highlevel.Array

In [470]:
type(mesh["2d", "triangles"].celldata.db)

awkward.highlevel.Array

`PointData` and `CellData` instances are actually represented by their wraooed data objects:

In [471]:
mesh["2d", "triangles"].celldata.db

<Array [{_nodes: [0, ...], _id: 0}, {...}] type='2 * {_nodes: 3 * int32, _i...'>

In the representation we can see the fields of the database. The fields are also accessible using the `fields` property of the data object:

In [472]:
mesh["2d", "triangles"].celldata.db.fields

['_nodes', '_id']

Field names starting with an underscore are internal variables crucial for the object to work properly. Overriding these fields might break the behaviour of the mesh. Besides these reserved field names, you can attach arbitrary data to the databases:

In [473]:
db = mesh["2d", "triangles"].celldata.db
number_of_cells = len(db)
db["random_data"] = np.random.rand(number_of_cells)

In [474]:
mesh["2d", "triangles"].celldata.db

<Array [{_nodes: [0, ...], _id: 0, ...}, ...] type='2 * {_nodes: 3 * int32,...'>

The newly attached data is now accessible as an Awkward array:

In [475]:
mesh["2d", "triangles"].celldata.db.random_data

<Array [0.793, 0.746] type='2 * float64'>

or a NumPy array

In [476]:
mesh["2d", "triangles"].celldata.db.random_data.to_numpy()

array([0.79271103, 0.74637708])

The data is also available like the database was a dictionary:

In [477]:
mesh["2d", "triangles"].celldata.db["random_data"]

<Array [0.793, 0.746] type='2 * float64'>

When bringing a mesh to a standard form, the Awkward library is smart enough to handle missing data. Let say we attach some random data to one of the point cloud of the mesh before briging it to standard form.

In [478]:
db = mesh["2d", "triangles"].pointdata.db
number_of_points = len(db)
db["random_data"] = np.random.rand(number_of_points)

In [479]:
mesh.to_standard_form()

PolyData({'2d': PolyData({'triangles': PolyData({}), 'quads': PolyData({})}), 'lines': PolyData({})})

In [480]:
mesh.pointdata.db

<Array [{random_data: 0.41, ...}, ..., {...}] type='12 * {random_data: floa...'>

You can turn `PointData` and `CellData` instances to other well known data formats (see the API reference for a full list of supported formats):

In [481]:
mesh.pointdata.to_dataframe(fields=["random_data"])

,random_data
entry,
0,0.409573
1,0.377465
2,0.757420
3,0.966008
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN


## Root, source and parent

In [482]:
mesh = generate_mesh()

The root is the top level `PolyData` instance in the mesh. The root of the root object is itself.

In [483]:
id(mesh), id(mesh.root()), id(mesh["2d", "triangles"].root())

(2436659693824, 2436659693824, 2436659693824)

To tell if an instance is a root or not use the `is_root` method:

In [484]:
mesh.is_root(), mesh["2d", "triangles"].is_root()

(True, False)

Every block of cells in a mesh -except the root- has a parent, which is the containing `PolyData` instance. The parent of the root instance is `None`.

In [485]:
id(mesh["2d"]), id(mesh["2d", "triangles"].parent)

(2436659695424, 2436659695424)

In [486]:
mesh.parent is None

True

In [487]:
mesh.is_root(), mesh["2d", "triangles"].is_root()

(True, False)

Every block with attached cell data has a source, that hosts the pointcloud the indices of the topology of the cells of the block are referring to.

In [488]:
id(mesh["2d", "triangles"]), id(mesh["2d", "triangles"].source())

(2436659694384, 2436659694384)

To tell if a `PolyData` hosts point related data, you can use the `is_source` method of the instance (remember that the mesh is decentralized at the moment):

In [489]:
mesh.is_source(), mesh["2d", "triangles"].is_source()

(False, True)

After bringing the mesh to a standard form:

In [490]:
mesh.to_standard_form()

PolyData({'2d': PolyData({'triangles': PolyData({}), 'quads': PolyData({})}), 'lines': PolyData({})})

In [491]:
mesh.is_source(), mesh["2d", "triangles"].is_source()

(True, False)